In [ ]:
import os
from glob import glob
import random

import pandas as pd
from tqdm.notebook import tqdm
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import joblib

import librosa

import warnings
warnings.filterwarnings('ignore') # to silence librosa warnings

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, GlobalAveragePooling1D, GlobalMaxPooling1D, Concatenate, LeakyReLU
from tensorflow.keras.optimizers.legacy import Adam, RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.utils import plot_model

tqdm.pandas()

import seaborn as sns

from triplet_dataset import TripletDataset

%load_ext dotenv
%dotenv

## Dataset

In [ ]:
class Track:
    def __init__(self, filepath: str, sr: int = 22050) -> None:
        self.filepath = filepath
        self.sr = sr
    
    def _normalize_mel_spectrogram(mel_spec: np.ndarray) -> np.ndarray:
        max_val = np.max(mel_spec)
        min_val = np.min(mel_spec)
        normalized_spectrogram = (mel_spec - min_val) / (max_val - min_val)

        return normalized_spectrogram
    
    def audio_extract(self, from_sec: int, to_sec: int) -> np.ndarray:
        audio, _ = librosa.load(
            self.filepath,
            mono=True,
            sr=self.sr,
            offset=from_sec,
            duration=to_sec - from_sec
        )

        if audio is None:
            raise Exception("Something went wrong went reading extract")

        return audio
    
    def spectrogram(self, from_sec: int = 40, to_sec: int = 43) -> np.ndarray:
        extract = self.audio_extract(from_sec, to_sec)

        spec = librosa.feature.melspectrogram(y=extract, sr=self.sr, n_fft=512, hop_length=128)
        spec_db = librosa.power_to_db(S=spec, ref=np.max)
        spec_db_norm = Track._normalize_mel_spectrogram(spec_db)

        return spec_db_norm

class TrackPair:
    def __init__(self, filepath_left: str, filepath_right: str, similar: bool) -> None:
        self.left = Track(filepath_left)
        self.right = Track(filepath_right)
        self.similar = similar

class Dataset:
    def __init__(self, triplets: TripletDataset) -> None:
        self.trackpairs: list[TrackPair] = []

        for _, row in triplets.df.iterrows():
            file_pair = (row["anchor"], row["positive"])
            track_pair = TrackPair(file_pair[0], file_pair[1], 1)
            self.trackpairs.append(track_pair)

            file_pair = (row["anchor"], row["negative"])
            track_pair = TrackPair(file_pair[0], file_pair[1], 0)
            self.trackpairs.append(track_pair)
    
    def as_dataframe(self) -> pd.DataFrame:
        data = {"left": [], "right": [], "similar": []}

        for track_pair in self.trackpairs:
            data["left"].append(track_pair.left.filepath)
            data["right"].append(track_pair.right.filepath)
            data["similar"].append(int(track_pair.similar))
        
        return pd.DataFrame(data)
    
    def as_training_data(self) -> tuple[np.ndarray, np.ndarray]:
        pairs = []
        labels = []

        for track_pair in tqdm(self.trackpairs):
            pairs.append([track_pair.left.spectrogram(), track_pair.right.spectrogram()])
            labels.append(track_pair.similar)
        
        return np.array(pairs).astype(float), np.array(labels)

In [ ]:
triplets = TripletDataset(os.environ["PATH_TO_TRACKS"], n = 2)
dataset = Dataset(triplets)

In [ ]:
def plot_track_spec(track: Track):
    plt.figure(figsize=(10, 2))

    librosa.display.specshow(track.spectrogram(), y_axis='mel')
    plt.colorbar()

    plt.tight_layout()
    plt.title(os.path.basename(track.filepath))
    plt.show()

def plot_spec(spec: np.ndarray):
    plt.figure(figsize=(10, 2))

    librosa.display.specshow(spec, y_axis='mel')
    plt.colorbar()

    plt.tight_layout()
    plt.show()

In [ ]:
for trackpair in dataset.trackpairs[:10]:
    print(f"Similar: {trackpair.similar}")
    plot_track_spec(trackpair.left)
    plot_track_spec(trackpair.right)

## Training data

In [ ]:
X, y = dataset.as_training_data()
y = np.float32(y)

print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")

## Build model

In [ ]:
#Check input data
print(f"Label: {y[0]}")
plot_spec(X[0][0])
plot_spec(X[0][1])

print(f"Label: {y[-7]}")
plot_spec(X[-7][0])
plot_spec(X[-7][1])

In [ ]:
input_shape = X.shape[2:]
input_shape

In [ ]:
class GlobalL2Pooling1D(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), axis=1))

leaky_relu_layer = LeakyReLU(alpha=0.01)

def euclidean_distance(embeddings):
    x, y = embeddings
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, _ = shapes
    return (shape1[0], 1)

def contrastive_loss_with_margin(margin):
    def contrastive_loss(y_true, y_pred):
        square_pred = K.square(y_pred)
        margin_square = K.square(K.maximum(margin - y_pred, 0))
        return (y_true * square_pred + (1 - y_true) * margin_square)
    return contrastive_loss

# Base network is from:
# (1) Recommending music on Spotify with deep learning. Sander Dieleman. https://sander.ai/2014/08/05/spotify-cnns.html (accessed 2024-03-23).
def build_base_network(input_shape):
    inputs = Input(shape=input_shape)

    x = Conv1D(filters=256, kernel_size=4, activation=leaky_relu_layer)(inputs)
    x = MaxPooling1D(pool_size=4)(x)

    x = Conv1D(filters=256, kernel_size=4, activation=leaky_relu_layer)(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(filters=512, kernel_size=4, activation=leaky_relu_layer)(x)

    # global temporal pooling
    mean_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    l2_pool = GlobalL2Pooling1D()(x)

    pooled_features = Concatenate()([mean_pool, max_pool, l2_pool])

    x = Dense(2048, activation=leaky_relu_layer)(pooled_features)
    x = Dense(2048, activation=leaky_relu_layer)(x)

    outputs = Dense(40)(x)

    return Model(inputs, outputs)

def build_siamese_network(base_network, input_shape):
    input_left = Input(shape=input_shape, name="input_left")
    input_right = Input(shape=input_shape, name="input_right")

    embeddings_left = base_network(input_left)
    embeddings_right = base_network(input_right)

    outputs = Lambda(euclidean_distance, name='output_layer',
        output_shape=eucl_dist_output_shape)([embeddings_left, embeddings_right])
    
    siamese_network = Model(inputs=[input_left, input_right], outputs=outputs)

    return siamese_network

base_network = build_base_network(input_shape)
model = build_siamese_network(base_network, input_shape)

model.compile(optimizer=Adam(learning_rate=0.00001), loss=contrastive_loss_with_margin(margin=1.0))

model.summary()

In [ ]:
base_network.summary()

## Train

In [ ]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [ ]:
input_shape

In [ ]:
history = model.fit(
    [X_train[:,0], X_train[:,1]],
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=([X_test[:,0], X_test[:,1]], y_test),
)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
train_dists = model.predict([X_train[:,0], X_train[:,1]])
test_dists = model.predict([X_test[:,0], X_test[:,1]])

In [ ]:
plt.subplot(2, 1, 1)
plt.title("Predicted distances Train")
sns.histplot(train_dists.ravel())
plt.xlim(0.0, 2.0)

plt.subplot(2, 1, 2)
plt.title("Predicted distances Test")
sns.histplot(test_dists.ravel())
plt.xlim(0.0, 2.0)

plt.tight_layout()
plt.show()

In [ ]:
very_similars = [spec for i, spec in enumerate(X_test) if test_dists[i] < 0.2]
for left, right in very_similars[:3]:
    print('================ Similar =================')
    plot_spec(left)
    plot_spec(right)

very_differents = [spec for i, spec in enumerate(X_test) if test_dists[i] > 1.2]
for left, right in very_differents[:3]:
    print('================ Different =================')
    plot_spec(left)
    plot_spec(right)

In [ ]:
def compute_accuracy(y_true, y_pred):
    return np.mean(y_pred == y_true)

y_pred_train = (train_dists < np.mean(test_dists)).ravel().astype(int)
train_accuracy = compute_accuracy(y_train, y_pred_train)

y_pred_val = (test_dists < np.mean(test_dists)).ravel().astype(int)
val_accuracy = compute_accuracy(y_test, y_pred_val)

print("Train Accuracy = {} Val accuracy = {}".format(train_accuracy, val_accuracy))

In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix, ConfusionMatrixDisplay

# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, 1 - test_dists)

plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_val)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def distance(left_path: str, right_path: str):
    spec_left = Track(left_path).spectrogram()
    spec_right = Track(right_path).spectrogram()

    pred = model.predict([
        spec_left.T.reshape(1, input_shape[0], input_shape[1]),
        spec_right.T.reshape(1, input_shape[0], input_shape[1])
    ], verbose=False)

    return pred.ravel()[0]

track_path_a = "/Users/leopnt/Music/TCOTC/Mostly James - Mostly James (TAN005) - 02 Do You Wanna Go-.aiff"
track_path_b = "/Users/leopnt/Music/TCOTC/Mostly James - Mostly James (TAN005) - 04 Do You Wanna Go- (SPF Remix).aiff"
track_path_c = "/Users/leopnt/Music/TCOTC/Dj Falcon & Thomas Bangalter - Together - 01 - Together.aiff"
track_path_d = "/Users/leopnt/Music/TCOTC/14 - Aneurysm.aiff"
track_path_e = "/Users/leopnt/Music/TCOTC/8 Bat Dream.aiff"
track_path_f = "/Users/leopnt/Music/TCOTC/05 - Abdul Raeva - Xpress [209645191].aiff"
track_path_g = "/Users/leopnt/Music/TCOTC/04. Cool Blue Liquid (Maximus Mix).aiff"
track_path_h = "/Users/leopnt/Music/TCOTC/04 Boney M. - Rasputin.aiff"
track_path_i = "/Users/leopnt/Music/TCOTC/03 - Tu dégages.aiff"

print("a vs e", distance(track_path_a, track_path_a))
print("a vs b", distance(track_path_a, track_path_b))
print("a vs c", distance(track_path_a, track_path_c))
print("a vs d", distance(track_path_a, track_path_d))
print("a vs e", distance(track_path_a, track_path_e))
print("a vs f", distance(track_path_a, track_path_f))
print("a vs g", distance(track_path_a, track_path_g))
print("a vs h", distance(track_path_a, track_path_h))
print("a vs i", distance(track_path_a, track_path_i))